In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize, minimize_scalar
import emcee
import matplotlib.pyplot as plt
from scipy.stats import norm, multivariate_normal
import seaborn as sns
import sys
sys.path.append("..")
from src.utils import contact_matrix
from src.dataset import CRKPTransmissionSimulator

from timeit import default_timer as timer

Computing the exact likelihood for the CRKP transmission model.

$$
    \log \mathcal{L}(\beta ; \mathbf{X}) = \sum_{t=2}^T \sum_{i=1}^N [(1 - X_{t-1}^{(i)})W_t^{(i)}W_{t-1}^{(i)}] \cdot [X_t^{(i)} \log (1 - e^{- \lambda_i(t)}) + (1 - X_t^{(i)}) \log e^{- \lambda_i(t)})]
$$

In [ ]:
path = "/Volumes/umms-esnitkin/Project_KPC_LTACH/Analysis/LTACH_transmission_modeling/preprocessed/resampled"

In [ ]:
X = pd.read_csv(f"{path}/infections.csv", index_col=0).values
W = pd.read_csv(f"{path}/facility_trace.csv", index_col=0).values
V = pd.read_csv(f"{path}/screening.csv", index_col=0).values
F = pd.read_csv(f"{path}/floor_trace.csv", index_col=0).values
R = pd.read_csv(f"{path}/room_trace.csv", index_col=0).values
cap = np.array([129., 28., 38., 35., 27., 17., 2])

In [ ]:
with open(f"{path}/observed_data.npy", "rb") as f:
            x_o = (np.load(f).T / cap).T

In [ ]:
def log_likelihood(beta, X, W, V, F, R, het=False):
    beta_0 = beta[0] if het else beta
    N, T = W.shape
    # X = np.empty((N, T))
    # load screen data for first day
    ans = 0
    for t in range(1, T):
        x = X[:, t-1]
        w = W[:, t-1]
        f = F[:, t-1]
        r = R[:, t-1]
        # who was infected at the last timestep?
        Ia = x[w > 0]
        hazard = Ia.sum() * beta_0 * np.ones(N) / cap[0]
        fa = f[w > 0]
        fC = contact_matrix(fa)
        # guarantee that there are no infecteds who aren't present
        # how many infected floormates?
        if het:
            hazard[w > 0] += (fC * Ia).sum(1) * beta[fa] / cap[fa]
        ra = r[w > 0]
        rC = contact_matrix(ra)
        infected_roommates = (rC * Ia).sum(1)
        if het:
            hazard[w > 0] += infected_roommates * beta[-1] / cap[-1]
        p = 1 - np.exp(-hazard)
        
        xt = X[:, t]
    
        valid = W[:, t] * w * (1 - x) # present for at least week & susceptible

        ans += np.nansum(valid * ((xt * np.log(p)) + (1 - xt) * np.log(1 - p)))

        # print(hazard[:3])
        # print(p[:3])

        # if t > 3: break

    return ans

# Maximum Likelihood Estimation

### homogeneous model

In [ ]:
def f(beta):
    return - log_likelihood(beta, X, W, V, F, R)

In [ ]:
res = minimize_scalar(f, bounds = (0.01, 1), method="bounded", options={"disp": True, "maxiter": 18})
res

### heterogeneous model

In [ ]:
def g(beta):
    return - log_likelihood(beta, X, W, V, F, R, True)

In [ ]:
res2 = minimize(g, x0=np.full(7, 0.05),
               bounds = [(0.0001, 1) for _ in range(7)], tol=0.0001)
res2

In [ ]:
np.round(res2.x, 3)

# MCMC

### homogeneous model

In [ ]:
def log_prob_homog(beta, X, W, V, F, R, prior_mu):
    prior = norm(prior_mu)
    lp = prior.logpdf(np.log(beta))
    # lp = log_prior(np.log(beta))
    return lp + log_likelihood(beta, X, W, V, F, R)

In [ ]:
nwalkers = 4
pos = res.x + 1e-4 * np.random.randn(nwalkers, 1)
prior_mu = -2
sampler = emcee.EnsembleSampler(
    nwalkers, 1, log_prob_homog, args = (X, W, V, F, R, prior_mu)
)
sampler.run_mcmc(pos, 2000, progress=True)

MCMC estimate uses 8,000 evaluations of the likelihood function.

In [ ]:
sampler.get_autocorr_time()

In [ ]:
plt.plot(sampler.get_chain()[:, :, 0], color="k", alpha=0.1)
plt.show()

In [ ]:
flat_samples = sampler.get_chain(discard=100, thin=50, flat=True)
flat_samples.shape

In [ ]:
print(np.log(flat_samples).mean())
print(np.log(flat_samples).std())
print(flat_samples.mean())

In [ ]:
plt.hist(flat_samples)
plt.show()

In [ ]:
# TODO: figure out how to benchmark homogeneous posterior predictive check

K = 30
T = 53
crkp_model = CRKPTransmissionSimulator(path, -3, 1, heterogeneous=False)
# neural_posterior_het = multivariate_normal(mu, sigma)
# npe_sample_het = neural_posterior_het.rvs(size=K, random_state=2)
posterior_predictive_mcmc = np.empty((K, 7, T))
for i in range(K):
    x_rep = crkp_model.CRKP_simulator(np.log(flat_samples[i]), i, summarize=True, show_full=True) * np.repeat(cap[:, None], T, 1) 
    posterior_predictive_mcmc[i] =  np.array(x_rep)

In [ ]:
x_rep[1:5].sum(0)

In [ ]:
for j in range(7):
    pp_mean = posterior_predictive_mcmc.mean(0)[j]
    labels = ["Post. Pred. Draw"] + [None for _ in range(K-1)]
    I_o = np.array(x_o)[j] * cap[j]
    plt.plot(I_o, label="Observed", color="k")
    plt.plot(posterior_predictive_mcmc[:, j, :].T, 
             label=labels, color="b",
             alpha=0.1)
    plt.plot(pp_mean, label="Post. Pred. Mean", linestyle="--",
             color="orange")
    # sns.lineplot(NN, color="green", linestyle="--", label="Total")
    plt.legend()
    if j == 6:
        ylab = "Infected Rooms"
    else:
        ylab = "Infected Patients"
    plt.ylabel(ylab)
    plt.xlabel("Weeks")
    # plt.savefig(f"images/crkp/crkp_ppc_abc{j}.png")
    plt.show()

### Heterogeneous Model

In [ ]:
def log_prob_het(logbeta, X, W, V, F, R, prior_mu):
    prior = multivariate_normal(np.full(7, prior_mu))
    lp = prior.logpdf(logbeta)
    return lp + log_likelihood(np.exp(logbeta), X, W, V, F, R, het=True)

In [ ]:
nwalkers = 16
initial = np.full(7, -3)
pos = initial + 1e-2 * np.random.randn(nwalkers, 7)
prior_mu = -3
sampler_het = emcee.EnsembleSampler(
    nwalkers, 7, log_prob_het, args = (X, W, V, F, R, prior_mu)
)
# up to 4,000
sampler_het.run_mcmc(pos, 5000, progress=True, skip_initial_state_check=True)

In [ ]:
plt.plot(sampler.get_chain()[:, :, 0], color="k", alpha=0.1)
plt.show()

In [ ]:
sampler.get_autocorr_time()

In [ ]:
flat_samples = sampler.get_chain(discard=500, thin=100, flat=True)
flat_samples.shape

In [ ]:
mcmc_pe = np.exp(flat_samples).mean(0)
mcmc_pe

In [ ]:
# convert to relative risks
w = mcmc_pe / cap 
w / w[0]

In [ ]:
flat_samples.mean(0)

In [ ]:
flat_samples.std(0)

In [ ]:
sns.heatmap(np.corrcoef(flat_samples.T))

## Posterior Predictive Checks

In [ ]:
flat_samples.shape

In [ ]:
K = 30
T = 53
crkp_model = CRKPTransmissionSimulator(path, -3, 1, heterogeneous=True)
# neural_posterior_het = multivariate_normal(mu, sigma)
# npe_sample_het = neural_posterior_het.rvs(size=K, random_state=2)
posterior_predictive_mcmc = np.empty((K, 7, T))
for i in range(K):
    x_rep = crkp_model.CRKP_simulator(flat_samples[i], i, summarize=True) * np.repeat(cap[:, None], T, 1) 
    posterior_predictive_mcmc[i] =  np.array(x_rep)

In [ ]:
for j in range(7):
    pp_mean = posterior_predictive_mcmc.mean(0)[j]
    labels = ["Post. Pred. Draw"] + [None for _ in range(K-1)]
    I_o = np.array(x_o)[j] * cap[j]
    plt.plot(I_o, label="Observed", color="k")
    plt.plot(posterior_predictive_mcmc[:, j, :].T, 
             label=labels, color="b",
             alpha=0.1)
    plt.plot(pp_mean, label="Post. Pred. Mean", linestyle="--",
             color="orange")
    # sns.lineplot(NN, color="green", linestyle="--", label="Total")
    plt.legend()
    if j == 6:
        ylab = "Infected Rooms"
    else:
        ylab = "Infected Patients"
    plt.ylabel(ylab)
    plt.xlabel("Weeks")
    # plt.savefig(f"images/crkp/crkp_ppc_abc{j}.png")
    plt.show()

# NPE

In [ ]:
# model.d_model=32,48,64,model.embed_dim=8,16,model.weight_decay=0,0.01,0.02

# best run: d_model = 48, d_embed = 8, weight_decay = 0
mu = -2.069
sigma = 0.163

# ABC Sanity Check

In [ ]:
def abc_rejection_sampler(S, epsilon, prior_sampler, simulator, 
                          x_o, max_attempts=10000, summarize=False,
                          print_every=5000, error_scaling = None):
    # S: total number of particles
    samples = []
    attempts = 0
    # x_o is shape (d_theta, d_x)
    errors = np.full(max_attempts, -1e3)
    start_time = timer()
    for s in range(S):
        accept = False
        while not accept:
            theta = np.array(prior_sampler())[0]
            x = simulator(theta, seed=attempts)
            accept, error = accept_sample(x, x_o, epsilon, summarize, error_scaling)
            if accept:
                samples.append(theta)
                # if s % 10 == 0: print(s)
            errors[attempts] = error
            attempts += 1
            if attempts == max_attempts:
                print("Maximum attempts reached, halting")
                return np.array(samples), errors
            if not attempts % print_every:
                print(f"Attempts: {attempts:,}")
    end_time = timer()
    accept_rate = S / attempts
    
    print(f"Time lapsed: {end_time - start_time:.2f} seconds")
    print(f"With tolerance {epsilon}, acceptance rate: {accept_rate:.6f}")
    print(f"Total number of attempts: {attempts:,}")
    return np.array(samples), errors

def accept_sample(x, x_o, epsilon, summarize, lam):
    if summarize:
        x_o = x_o[0]
    v = np.array(x - x_o)
    if lam is not None:
        v = v * lam.reshape(-1, 1)
    error = np.linalg.norm(v)
    accept = (error < epsilon)

    return accept, error

In [ ]:
summarize = False #True
hetero = True

prior_mu = -2
prior_sigma = 1
model = CRKPTransmissionSimulator(path, prior_mu, prior_sigma,
                                  heterogeneous=False)


prior_sampler = lambda: model.sample_logbeta(1)
simulator = lambda theta, seed:(model.CRKP_simulator(theta, seed))[W > 0]
# simulator = lambda theta, seed: np.nansum(model.CRKP_simulator(theta, seed), 0)

In [ ]:
obs = X[W > 0] # np.nansum(X, 0)
abc_samples, error = abc_rejection_sampler(100, 20, prior_sampler, simulator, 
                          obs, max_attempts=1000000, summarize=False,
                          print_every=1000, error_scaling = None)

In [ ]:
np.exp(abc_samples[:, 0]).mean()

In [ ]:
(X == 1).any(1).sum()

In [ ]:
abc_samples[:, 0][0]

In [ ]:
np.log(0.025)

In [ ]:
I_o = np.nansum(X, 0)

K = 30
T = 53
crkp_model = CRKPTransmissionSimulator(path, -2, 1, heterogeneous=False)
# neural_posterior_het = multivariate_normal(mu, sigma)
# npe_sample_het = neural_posterior_het.rvs(size=K, random_state=2)
posterior_predictive_abc = np.empty((K, T))
for i in range(K):
    # x_rep = crkp_model.CRKP_simulator(abc_samples[:, 0][i], i)
    x_rep = crkp_model.CRKP_simulator(np.log(.126), i)
    i_rep = np.nansum(x_rep, 0)
    posterior_predictive_abc[i] =  np.array(i_rep)

In [ ]:
pp_mean = posterior_predictive_abc.mean(0)
plt.plot(I_o, label="Observed", color="k")
plt.plot(posterior_predictive_abc.T, 
         label="Post. Pred. Draw", color="b",
         alpha=0.1)
plt.plot(pp_mean, label="Post. Pred. Mean", linestyle="--",
         color="orange")
plt.show()